# 夏目漱石のテキストでGPT訓練

このノートブックでは、夏目漱石のテキストを使用してGPTモデルを訓練します。
nanoGPTと同様のアプローチで、文字レベルの言語モデルを作成します。

**Google Colabでの実行:** このノートブックはGoogle Colabで直接実行できます。必要なコードはすべてノートブック内に含まれており、追加のパッケージインストールは不要です。

## GPU ランタイムの設定（Google Colab）

このノートブックではGPTモデルの訓練を行うため、**GPUランタイムの使用を推奨します**。
Google Colabの無料版（T4 GPU）で十分に動作します。

**設定手順：**
1. メニューから「ランタイム」→「ランタイムのタイプを変更」を選択
2. 「ハードウェア アクセラレータ」で「T4 GPU」を選択
3. 「保存」をクリック

> CPUでも実行可能ですが、訓練に大幅に時間がかかります。

## 必要なライブラリのインポート

In [ ]:
import os
import re
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import requests
from bs4 import BeautifulSoup
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

## 夏目漱石のテキストダウンロード関数

青空文庫から夏目漱石の「吾輩は猫である」をダウンロードし、テキストを前処理します。

In [ ]:
def download_soseki():
    """夏目漱石のテキストをダウンロード。"""
    url = "https://www.aozora.gr.jp/cards/000148/files/789_14547.html"

    # データディレクトリを作成
    os.makedirs("data/soseki", exist_ok=True)

    # ダウンロードして処理
    filepath = "data/soseki/input.txt"
    if not os.path.exists(filepath):
        print("夏目漱石のテキストをダウンロード中...")
        response = requests.get(url)
        response.encoding = 'shift_jis'  # 青空文庫はShift_JIS

        # BeautifulSoupでHTMLを解析
        soup = BeautifulSoup(response.text, 'html.parser')

        # 本文を抽出（青空文庫の構造に基づく）
        main_text = soup.find('div', class_='main_text')
        if main_text:
            text = main_text.get_text()
        else:
            # フォールバック：bodyからテキストを抽出
            text = soup.get_text()

        # 青空文庫の注記や記号を除去
        text = re.sub(r'［＃.*?］', '', text)  # 注記を除去
        text = re.sub(r'《.*?》', '', text)    # ルビを除去
        text = re.sub(r'｜', '', text)        # 縦線を除去
        text = re.sub(r'　', ' ', text)       # 全角スペースを半角に
        text = re.sub(r'\n+', '\n', text)     # 連続する改行を一つに
        text = text.strip()

        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"ダウンロード完了: {filepath}")

    # テキストを読み込み
    with open(filepath, 'r', encoding='utf-8') as f:
        text = f.read()

    print(f"データセットサイズ: {len(text)} 文字")
    return text

## データの取得

夏目漱石のテキストをダウンロードして確認します。

In [ ]:
# データセットをダウンロードして読み込み
text = download_soseki()

# テキストの一部を表示
print("テキストの最初の500文字:")
print(text[:500])

## トークナイザーの実装

文字レベルのトークナイザーを実装します。テキスト中のユニークな文字を語彙とし、文字とIDの相互変換を行います。

In [ ]:
class SimpleTokenizer:
    """文字レベルのテキスト処理用トークナイザ."""

    def __init__(self, text):
        chars = sorted(list(set(text)))
        self.vocab_size = len(chars)
        self.char_to_idx = {ch: i for i, ch in enumerate(chars)}
        self.idx_to_char = {i: ch for ch, i in self.char_to_idx.items()}

    def encode(self, text):
        return [self.char_to_idx.get(ch, 0) for ch in text]

    def decode(self, tokens):
        return ''.join([self.idx_to_char.get(int(idx), '') for idx in tokens])


# トークナイザーを作成
tokenizer = SimpleTokenizer(text)
print(f"語彙数: {tokenizer.vocab_size} ユニーク文字")

# 語彙の一部を表示
print("\n語彙の一部:")
chars = list(tokenizer.char_to_idx.keys())[:20]
print(chars)

## データセットとデータローダーの実装

自己回帰型の言語モデル学習のため、テキストを入力・ターゲットのペアに変換するデータセットクラスとデータローダーを実装します。

In [ ]:
class TextDataset(Dataset):
    """テキストデータセットクラス"""

    def __init__(self, text, tokenizer, block_size=128):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.tokens = tokenizer.encode(text)
        print(f"データセットサイズ: {len(self.tokens)} トークン")

    def __len__(self):
        return len(self.tokens) - self.block_size

    def __getitem__(self, idx):
        # 入力とターゲットのペアを作成
        chunk = self.tokens[idx:idx + self.block_size + 1]
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y


def create_dataloaders(text, tokenizer, block_size=128, batch_size=64,
                       train_split=0.9, num_workers=0):
    """学習用と検証用のデータローダーを作成する."""
    dataset = TextDataset(text, tokenizer, block_size)
    n = len(dataset)
    n_train = int(train_split * n)
    n_val = n - n_train
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [n_train, n_val]
    )
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=torch.cuda.is_available()
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=torch.cuda.is_available()
    )
    return train_loader, val_loader


# データローダーを作成
train_loader, val_loader = create_dataloaders(
    text, tokenizer,
    block_size=256,      # コンテキスト長
    batch_size=64,       # バッチサイズ
    train_split=0.9
)

print(f"訓練データバッチ数: {len(train_loader)}")
print(f"検証データバッチ数: {len(val_loader)}")

## GPT-2標準設定の参考

**GPT-2の各モデルサイズの仕様:**

| モデル | パラメータ | n_embd | n_layer | n_head | vocab_size |
|--------|------------|--------|---------|--------|------------|
| Small  | 124M       | 768    | 12      | 12     | 50257      |
| Medium | 355M       | 1024   | 24      | 16     | 50257      |
| Large  | 774M       | 1280   | 36      | 20     | 50257      |
| XL     | 1.5B       | 1600   | 48      | 25     | 50257      |

**推奨設定:**
- **リソース限定**: n_embd=512, n_layer=8, n_head=8 (~40M params)
- **標準学習**: GPT-2 Small設定 (124M params)
- **本格学習**: GPT-2 Medium以上

下記では軽量な設定を使用します。

## アテンション機構（2章で実装したモジュールの再利用）

GPTモデルは、2章で実装したマルチヘッドアテンション機構を再利用します。
スケール内積アテンション → アテンションヘッド → マルチヘッドアテンションの順に定義します。

In [ ]:
class ScaledDotProductAttention(nn.Module):
    """スケール内積アテンション（2章で実装）"""

    def forward(self, query, key, value, mask=None):
        d_k = query.size(-1)
        score = torch.bmm(query, key.transpose(1, 2)) / (d_k ** 0.5)
        if mask is not None:
            score = score.masked_fill(mask, float("-inf"))
        weight = torch.softmax(score, dim=-1)
        output = torch.bmm(weight, value)
        return output


class AttentionHead(nn.Module):
    """マルチヘッドアテンションの単一ヘッド（2章で実装）"""

    def __init__(self, d_k, d_v, d_model):
        super().__init__()
        self.linear_q = nn.Linear(d_model, d_k)
        self.linear_k = nn.Linear(d_model, d_k)
        self.linear_v = nn.Linear(d_model, d_v)
        self.attention = ScaledDotProductAttention()

    def forward(self, query, key, value, mask=None):
        query = self.linear_q(query)
        key = self.linear_k(key)
        value = self.linear_v(value)
        return self.attention(query, key, value, mask=mask)


class MultiHeadAttention(nn.Module):
    """マルチヘッドアテンション（2章で実装）"""

    def __init__(self, n_heads, d_k, d_v, d_model):
        super().__init__()
        self.heads = nn.ModuleList(
            [AttentionHead(d_k, d_v, d_model) for _ in range(n_heads)]
        )
        self.linear_o = nn.Linear(n_heads * d_v, d_model)

    def forward(self, query, key, value, mask=None):
        head_out = [head(query, key, value, mask=mask) for head in self.heads]
        head_out = torch.cat(head_out, dim=-1)
        return self.linear_o(head_out)


print("アテンション機構を定義しました")

## GPTモデルのアーキテクチャ

GPT-2のアーキテクチャを実装します。2章のTransformerからの主な変更点：
- **学習可能な位置埋め込み**: 固定の正弦波方式に代わり、`nn.Embedding`で実装
- **Pre-Layer Normalization**: 各サブレイヤーの前に`nn.LayerNorm`を配置
- **因果的マスク（Causal Mask）**: 未来のトークンへのアテンションを防止
- **デコーダーのみ**: エンコーダーとソース・ターゲットアテンションを削除

In [ ]:
class GPTMultiHeadAttention(nn.Module):
    """GPT用のマルチヘッドアテンション（causal mask付き）"""

    def __init__(self, n_embd, n_head, dropout=0.1):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        self.n_embd = n_embd
        d_k = d_v = n_embd // n_head
        self.attention = MultiHeadAttention(n_head, d_k, d_v, n_embd)
        self.resid_dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.size()
        # causal mask を作成（未来のトークンへの注意を防ぐ）
        causal_mask = torch.triu(
            torch.ones(T, T, device=x.device), diagonal=1
        ).bool()
        causal_mask = causal_mask.unsqueeze(0).expand(B, -1, -1)
        y = self.attention(x, x, x, mask=causal_mask)
        y = self.resid_dropout(y)
        return y


class TransformerBlock(nn.Module):
    """Transformerブロック（Pre-LN方式）"""

    def __init__(self, n_embd, n_head, dropout=0.1):
        super().__init__()
        self.ln_1 = nn.LayerNorm(n_embd)
        self.attn = GPTMultiHeadAttention(n_embd, n_head, dropout)
        self.ln_2 = nn.LayerNorm(n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


class GPT(nn.Module):
    """GPTモデルの基本構造"""

    def __init__(self, vocab_size, n_embd=768, n_layer=12, n_head=12,
                 block_size=1024, dropout=0.1):
        super().__init__()
        self.block_size = block_size
        self.n_embd = n_embd
        # トークンと位置の埋め込み
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(block_size, n_embd)
        self.drop = nn.Dropout(dropout)
        # Transformerブロック（n_layer個）
        self.blocks = nn.Sequential(*[
            TransformerBlock(n_embd, n_head, dropout)
            for _ in range(n_layer)
        ])
        # 最終層の正規化と出力層
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)
        # 重みの初期化
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.ones_(module.weight)
            torch.nn.init.zeros_(module.bias)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # トークンと位置の埋め込みを加算
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device).unsqueeze(0)
        tok_emb = self.token_embedding(idx)
        pos_emb = self.position_embedding(pos)
        x = self.drop(tok_emb + pos_emb)
        # Transformerブロックを通す
        x = self.blocks(x)
        x = self.ln_f(x)
        # 語彙への投影
        logits = self.head(x)
        # 損失の計算（学習時のみ）
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """テキストを自己回帰的に生成する."""
        for _ in range(max_new_tokens):
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


class GPTConfig:
    """GPTモデルの設定クラス"""

    def __init__(self, **kwargs):
        self.vocab_size = kwargs.get('vocab_size', 50257)
        self.n_embd = kwargs.get('n_embd', 768)
        self.n_layer = kwargs.get('n_layer', 12)
        self.n_head = kwargs.get('n_head', 12)
        self.block_size = kwargs.get('block_size', 1024)
        self.dropout = kwargs.get('dropout', 0.1)

    def get_model_size(self):
        """モデルのパラメータ数を概算する（百万単位）."""
        params = self.vocab_size * self.n_embd
        params += self.block_size * self.n_embd
        params_per_block = (
            self.n_head * (self.n_embd // self.n_head) * self.n_embd * 3 +
            self.n_head * (self.n_embd // self.n_head) * self.n_embd +
            self.n_embd * 4 * self.n_embd +
            4 * self.n_embd * self.n_embd +
            self.n_embd * 4
        )
        params += params_per_block * self.n_layer
        params += self.n_embd * 2
        params += self.n_embd * self.vocab_size
        return params / 1e6


print("GPTモデルを定義しました")

## モデル設定とモデル作成

nanoGPTのshakespeareと同様の設定でGPTモデルを作成します。

In [ ]:
# モデル設定（nanoGPTのshakespeareと同様）
config = GPTConfig(
    vocab_size=tokenizer.vocab_size,
    n_embd=384,          # 埋め込み次元
    n_layer=6,           # レイヤー数
    n_head=6,            # アテンションヘッド数
    block_size=256,      # コンテキストウィンドウ
    dropout=0.2
)

print(f"モデルサイズ: 約{config.get_model_size():.2f}Mパラメータ")

# モデルを作成
model = GPT(
    vocab_size=config.vocab_size,
    n_embd=config.n_embd,
    n_layer=config.n_layer,
    n_head=config.n_head,
    block_size=config.block_size,
    dropout=config.dropout
)

print("モデル作成完了")

## トレーナーの実装

AdamWオプティマイザーと学習率スケジューリング（線形ウォームアップ＋コサイン減衰）を使用するトレーナーを実装します。Weight decayはバイアス項・LayerNorm・埋め込み層には適用しません。

In [ ]:
class GPTTrainer:
    """GPTモデル用のトレーナークラス."""

    def __init__(self, model, train_loader, val_loader,
                 learning_rate=3e-4, weight_decay=0.1,
                 warmup_steps=1000, max_steps=10000,
                 grad_clip=1.0, device=None):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.max_steps = max_steps
        self.warmup_steps = warmup_steps
        self.grad_clip = grad_clip

        if device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = device
        self.model = self.model.to(self.device)

        self.optimizer = self._configure_optimizer(learning_rate, weight_decay)
        self.scheduler = CosineAnnealingLR(
            self.optimizer, T_max=max_steps - warmup_steps
        )

    def _configure_optimizer(self, learning_rate, weight_decay):
        """重み減衰付きAdamWオプティマイザを設定する."""
        decay_params = []
        no_decay_params = []
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if 'bias' in name or 'ln' in name or 'embedding' in name:
                    no_decay_params.append(param)
                else:
                    decay_params.append(param)
        optimizer_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': no_decay_params, 'weight_decay': 0.0}
        ]
        return AdamW(optimizer_groups, lr=learning_rate, betas=(0.9, 0.95))

    def get_lr(self, step):
        if step < self.warmup_steps:
            return self.optimizer.param_groups[0]['lr'] * step / self.warmup_steps
        return self.optimizer.param_groups[0]['lr']

    @torch.no_grad()
    def evaluate(self, max_batches=10):
        """検証セットでモデルを評価する."""
        self.model.eval()
        losses = []
        for i, (x, y) in enumerate(self.val_loader):
            if i >= max_batches:
                break
            x, y = x.to(self.device), y.to(self.device)
            _, loss = self.model(x, y)
            losses.append(loss.item())
        self.model.train()
        return np.mean(losses) if losses else float('inf')

    def train(self, log_interval=100, eval_interval=500):
        """メイン学習ループ."""
        self.model.train()
        train_losses = []
        val_losses = []
        step = 0
        epoch = 0

        print(f"学習デバイス: {self.device}")
        print(f"モデルパラメータ数: {sum(p.numel() for p in self.model.parameters())/1e6:.2f}M")
        start_time = time.time()

        while step < self.max_steps:
            epoch += 1
            for x, y in self.train_loader:
                if step >= self.max_steps:
                    break
                x, y = x.to(self.device), y.to(self.device)

                # 順伝播・逆伝播
                logits, loss = self.model(x, y)
                self.optimizer.zero_grad()
                loss.backward()
                if self.grad_clip > 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                self.optimizer.step()
                train_losses.append(loss.item())

                if step >= self.warmup_steps:
                    self.scheduler.step()

                if step % log_interval == 0:
                    avg_loss = np.mean(train_losses[-log_interval:]) if len(train_losses) >= log_interval else loss.item()
                    elapsed = time.time() - start_time
                    print(f"Step {step}/{self.max_steps} | Loss: {avg_loss:.4f} | "
                          f"LR: {self.get_lr(step):.6f} | Time: {elapsed:.1f}s")

                if step % eval_interval == 0 and step > 0:
                    val_loss = self.evaluate()
                    val_losses.append(val_loss)
                    print(f"検証損失: {val_loss:.4f}")

                step += 1

        print(f"学習完了！総時間: {time.time() - start_time:.1f}s")
        return {'train_losses': train_losses, 'val_losses': val_losses}

    def save_checkpoint(self, path):
        """モデルチェックポイントを保存する."""
        checkpoint = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }
        torch.save(checkpoint, path)
        print(f"チェックポイントを{path}に保存しました")


print("GPTTrainerを定義しました")

## トレーナーの設定

モデルの訓練に必要なトレーナーを設定します。

In [ ]:
# トレーナーを作成
trainer = GPTTrainer(
    model, train_loader, val_loader,
    learning_rate=1e-3,
    weight_decay=0.1,
    warmup_steps=100,
    max_steps=5000,      # 5000ステップ訓練
    grad_clip=1.0
)

print("トレーナー設定完了")
print(f"デバイス: {trainer.device}")

## モデルの訓練

GPTモデルを夏目漱石のテキストで訓練します。

In [ ]:
# モデルを訓練
print("訓練開始...")
losses = trainer.train(log_interval=100, eval_interval=500)
print("訓練完了!")

## テキスト生成のテスト

訓練されたモデルを使用して、夏目漱石風のテキストを生成します。

In [ ]:
print("夏目漱石風のテキストを生成中...")
print("=" * 50)

model.eval()

# プロンプトで開始
prompts = [
    "吾輩は",
    "夏の"
]

for prompt in prompts:
    print(f"\nプロンプト: {prompt}")
    print("-" * 30)
    
    # プロンプトをエンコード
    prompt_tokens = torch.tensor(
        tokenizer.encode(prompt),
        dtype=torch.long
    ).unsqueeze(0).to(trainer.device)
    
    # テキストを生成
    with torch.no_grad():
        generated = model.generate(
            prompt_tokens,
            max_new_tokens=200,
            temperature=0.8,
            top_k=40
        )
    
    generated_text = tokenizer.decode(generated[0].cpu().numpy())
    print(generated_text)

## モデルの保存

訓練されたモデルを保存します。

In [ ]:
# モデルを保存
os.makedirs("models", exist_ok=True)
checkpoint_path = "models/soseki_gpt_checkpoint.pt"
trainer.save_checkpoint(checkpoint_path)
print(f"モデル保存: {checkpoint_path}")

## 訓練損失の可視化（オプション）

訓練過程の損失をプロットして確認します。

In [ ]:
import matplotlib.pyplot as plt

# 損失をプロット
if losses:
    train_losses = losses['train_losses']
    val_losses = losses['val_losses']
    
    plt.figure(figsize=(12, 4))
    
    # 訓練損失
    plt.subplot(1, 2, 1)
    plt.plot(train_losses)
    plt.title('Training Loss')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.grid(True)
    
    # 検証損失
    if val_losses:
        plt.subplot(1, 2, 2)
        plt.plot(val_losses)
        plt.title('Validation Loss')
        plt.xlabel('Evaluation Step')
        plt.ylabel('Loss')
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("損失データが見つかりません")

## まとめ

このノートブックでは以下を実行しました：

1. 夏目漱石のテキストを青空文庫からダウンロード
2. テキストの前処理（注記・ルビの除去など）
3. 文字レベルトークナイザーの作成
4. データローダーの作成
5. GPTモデルの設定と作成
6. モデルの訓練
7. テキスト生成のテスト
8. モデルの保存

訓練されたモデルは夏目漱石のスタイルを学習し、類似したテキストを生成できるようになります。